In [ ]:
import torch 
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
from torch.utils.data import TensorDataset
from torch.utils.data import random_split

In [ ]:
input_size = 43
learning_rate = 0.001
num_epochs = 100

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.DataFrame()
df = pd.read_csv('./train_data.csv', encoding = 'utf-8')
df["Target"]= df["Target"].astype(str)

In [ ]:
df.head()

,Unnamed: 0,Target,Embedding,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44
0,0,BJP,0.610007,-1.373251,-0.477155,0.542287,-0.330684,0.521968,1.343305,-0.772400,0.354372,-1.105608,1.000000,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001
1,1,BJP,1.132800,0.431400,0.006784,0.733220,1.157032,-0.967515,-0.248303,0.397667,-0.720171,1.133229,0.666667,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001
2,2,AAP,0.488085,-1.489697,0.688916,0.512605,-0.455515,1.563607,0.577907,-1.479291,0.941073,0.194777,1.000000,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001
3,3,AAP,-0.588054,-0.868341,-0.711547,-0.672294,0.710237,-0.237520,-0.151698,-0.111626,1.718842,0.096721,2.000000,0.000,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001
4,4,AAP,0.454455,0.485844,-0.169238,0.695180,0.764226,0.006655,-2.008995,-1.131305,-0.226778,-0.420710,0.001000,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001


In [ ]:
label = df.pop('Target')
label= label.astype(str)
print(label)

0           BJP
1           BJP
2           AAP
3           AAP
4           AAP
         ...   
477    Congress
478         BJP
479         BJP
480       AIMIM
481         BJP
Name: Target, Length: 482, dtype: object


In [ ]:
df.pop('Unnamed: 0')

0        0
1        1
2        2
3        3
4        4
      ... 
477    477
478    478
479    479
480    480
481    481
Name: Unnamed: 0, Length: 482, dtype: int64

In [ ]:
train_data = torch.tensor(df.values).type(torch.FloatTensor)
train_data = torch.reshape(train_data,[482,1,1,43])
train_data.size()

torch.Size([482, 1, 1, 43])

In [ ]:
label = pd.DataFrame(label)
type(label)

pandas.core.frame.DataFrame

In [ ]:
labels = np.array(label.values)

In [ ]:
from sklearn import preprocessing
le = preprocessing.OneHotEncoder()
label = le.fit_transform(label.values)

In [ ]:
label = torch.from_numpy(label.todense())

In [ ]:
print(label)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)


In [ ]:
dataset = TensorDataset(train_data,label)

In [ ]:
torch.manual_seed(42)

In [ ]:
train_ds, test_ds = random_split(dataset,[385,97])

In [ ]:
train_dl = DataLoader(train_ds,32,shuffle=True,num_workers=3,pin_memory=True)
test_dl = DataLoader(test_ds,32,shuffle=True,num_workers=3,pin_memory=True)
total_dl = DataLoader(dataset,32,shuffle=True,num_workers=3,pin_memory=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class CNN(nn.Module):
  def __init__(self, in_channel = 1, num_classes = 29):
    super(CNN, self).__init__()
    self.leakyR = nn.LeakyReLU(0.2)
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size= (2,2), stride= (1,1), padding= (1,1))
    self.pool = nn.MaxPool2d(kernel_size= (2,2), stride= (2,2))
    self.conv2 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size= (2,2), stride= (1,1), padding= (1,1))
    self.fc1 = nn.Linear(176, num_classes)
    #self.hl = nn.Linear(150, num_classes)

  def forward(self, x):
    x = self.leakyR(self.conv1(x))
    #print("conv1: ")
    #print(x.size())
    x = self.pool(x)
    #print("pool: ")
    #print(x.size())
    x = self.leakyR(self.conv2(x))
    #print("conv2 ")
    #print(x.size())
    x = self.pool(x)
    #print("pool: ")
    #print(x.size())
    x = x.reshape(x.shape[0], -1)
    #print("reshape: ")
    #print(x.size())
    x = self.fc1(x)
    #print("fc1: ")
    #print(x.size())
    x = F.softmax(x,dim=1)
    #print("softmax: ")
    #print(x.size())
    return x  

model = CNN()

optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate, betas= (0.5, 0.999))
#optimizer =  torch.optim.SGD(model.parameters(), lr= learning_rate)


In [ ]:
for epoch in range(num_epochs):
  model.train()
  for batch_idx, (data, targets) in enumerate(train_dl):
    scores = model(data)
    loss = F.cross_entropy(scores, torch.argmax(targets,dim=1))
    loss = loss.float()
    optimizer.zero_grad()
    loss.backward()
    
    optimizer.step()



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
@torch.no_grad()
def accuracy(train_dl):
  acc = 0;
  cnt =0;
  for batch,labels in train_dl:
    model.eval()
    out = model(batch)
    preds = torch.argmax(out,dim=1)
    label = torch.argmax(labels,dim=1)
    cnt+=1
    acc += torch.tensor(torch.sum(preds == label).item()/len(preds)).item()
  accuracy = acc/cnt
  return accuracy

In [ ]:
#torch.save(model.state_dict(),'good_model.pth')

In [ ]:
accuracy(train_dl)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


0.49759615384615385

In [ ]:
accuracy(test_dl)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


0.3125

In [ ]:
accuracy(total_dl)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


0.451171875